In [1]:
### Installing the required packages if not already installed
packages = ['numpy', 'pandas', 'warnings', 'sqlite3', 'yfinance', 'numba', 'time']

for package in packages:
    try:
        __import__(package)
    except ImportError:
        %pip install {package}


### Start timer
import time
start = time.time()

import numpy as np
import pandas as pd
import warnings
import sqlite3
import os

from numba import njit

### Ignoring the warnings
warnings.filterwarnings('ignore')

### Setting working directory
os.chdir('/Users/emilwilliamhansen/Desktop/Master-Thesis/Code')

In [2]:
### Reading the datasets
monthly_80_20 = (pd.read_csv("Data/obi/monthly_stock_returns_ose.csv",
                      sep=';', encoding='latin1')[["ISIN", "ticker", "Last_Sec_Name", "Date", "MonthlyReturn", "LastPrice", "NoShares"]]
                      ).rename(columns={"Last_Sec_Name": "Name", "MonthlyReturn": "Return", "LastPrice": "Price", "NoShares": "Shares"})

monthly_20_24 = (pd.read_csv('Data/obi/ose_equity_euronext_data/monthly_ose_stocks_nov_2020_aug_2024.csv',
                         sep=',', encoding='latin1')[["ISIN", "ticker", "Name", "Date", "Return", "Price", "SharesOutstanding"]]
                         ).rename(columns={"SharesOutstanding": "Shares"})

### Combining the datasets
monthly = pd.concat([monthly_80_20, monthly_20_24])
del monthly_80_20, monthly_20_24

### Fixing the date format
monthly["Date"] = pd.to_datetime(monthly["Date"], format="%Y%m%d")

monthly

,ISIN,ticker,Name,Date,Return,Price,Shares
0,NO0003069908,AAT,Aust-Agder Trafikkselskap,1980-01-31,0.0000,240.00,4000.0
1,NO0003069908,AAT,Aust-Agder Trafikkselskap,1980-02-29,0.0000,240.00,4000.0
2,NO0003069908,AAT,Aust-Agder Trafikkselskap,1980-03-31,0.0000,240.00,4000.0
3,NO0003069908,AAT,Aust-Agder Trafikkselskap,1980-04-30,0.0417,250.00,4000.0
4,NO0003069908,AAT,Aust-Agder Trafikkselskap,1980-05-31,0.4600,325.00,4000.0
...,...,...,...,...,...,...,...
14953,US36467X2062,GIG,GAMING INNOVATION,2024-08-31,-0.0334,28.95,127132040.0
14954,VGG3175Q1081,EPIC,Epic Gas,2020-12-31,0.0191,16.00,106616352.0
14955,VGG3175Q1081,EPIC,Epic Gas,2021-01-31,-0.0437,15.30,106616352.0
14956,VGG3175Q1081,EPIC,EPIC GAS,2021-02-28,0.1432,16.30,106616352.0


In [3]:
daily_80_90 = (pd.read_csv("Data/obi/daily_stock_returns_ose_csv/daily_stock_returns_ose_1980_1989.csv",
                        sep=';', encoding='latin1')[["ISIN", "ticker", "Last_Sec_Name", "Date", "Return", "ClosePrice", "SharesOutstanding"]]
                        ).rename(columns={"Last_Sec_Name": "Name", "ClosePrice": "Price", "SharesOutstanding": "Shares"})

daily_90_00 = (pd.read_csv("Data/obi/daily_stock_returns_ose_csv/daily_stock_returns_ose_1990_1999.csv",
                        sep=';', encoding='latin1')[["ISIN", "ticker", "Last_Sec_Name", "Date", "Return", "ClosePrice", "SharesOutstanding"]]
                        ).rename(columns={"Last_Sec_Name": "Name", "ClosePrice": "Price", "SharesOutstanding": "Shares"})

daily_00_10 = (pd.read_csv("Data/obi/daily_stock_returns_ose_csv/daily_stock_returns_ose_2000_2009.csv",
                        sep=';', encoding='latin1')[["ISIN", "ticker", "Last_Sec_Name", "Date", "Return", "ClosePrice", "SharesOutstanding"]]
                        ).rename(columns={"Last_Sec_Name": "Name", "ClosePrice": "Price", "SharesOutstanding": "Shares"})

daily_10_20 = (pd.read_csv("Data/obi/daily_stock_returns_ose_csv/daily_stock_returns_ose_2010_2020.csv",
                        sep=';', encoding='latin1')[["ISIN", "ticker", "Last_Sec_Name", "Date", "Return", "ClosePrice", "SharesOutstanding"]]
                        ).rename(columns={"Last_Sec_Name": "Name", "ClosePrice": "Price", "SharesOutstanding": "Shares"})

daily_20_24 = (pd.read_csv('Data/obi/ose_equity_euronext_data/daily_ose_stocks_nov_2020_aug_2024.csv',
                         sep=';', encoding='latin1')[["ticker", "Name", "Date", "Return", "ClosePrice", "SharesOutstanding"]]
                        ).rename(columns={"ClosePrice": "Price", "SharesOutstanding": "Shares"})

### Fixing the ISIN values for 2020-2024 dataset by extracing the ISIN values from the monthly dataset
daily_20_24['ISIN'] = np.nan
daily_20_24 = daily_20_24.merge(monthly[["ISIN", "ticker"]], on="ticker", how="left").drop(columns='ISIN_x').rename(columns={"ISIN_y": "ISIN"})

### Combining the datasets
daily = pd.concat([daily_80_90, daily_90_00, daily_00_10, daily_10_20, daily_20_24])
del daily_80_90, daily_90_00, daily_00_10, daily_10_20, daily_20_24


### Fixing the date format
daily["Date"] = pd.to_datetime(daily["Date"], format="%Y%m%d")

daily

,ISIN,ticker,Name,Date,Return,Price,Shares
0,NO0003069908,AAT,Aust-Agder Trafikkselskap,1980-01-04,0.0000,NaN,4000.0
1,NO0003069908,AAT,Aust-Agder Trafikkselskap,1980-01-08,0.0000,NaN,4000.0
2,NO0003069908,AAT,Aust-Agder Trafikkselskap,1980-01-11,0.0000,NaN,4000.0
3,NO0003069908,AAT,Aust-Agder Trafikkselskap,1980-01-15,0.0000,NaN,4000.0
4,NO0003069908,AAT,Aust-Agder Trafikkselskap,1980-01-18,0.0000,NaN,4000.0
...,...,...,...,...,...,...,...
35407819,VGG3175Q1081,EPIC,EPIC GAS,2021-03-04,-0.0338,14.3,106616352.0
35407820,VGG3175Q1081,EPIC,EPIC GAS,2021-03-05,-0.0350,13.8,106616352.0
35407821,VGG3175Q1081,EPIC,EPIC GAS,2021-03-05,-0.0350,13.8,106616352.0
35407822,VGG3175Q1081,EPIC,EPIC GAS,2021-03-05,-0.0350,13.8,106616352.0


In [4]:
### Getting the values unique rows for ISIN and ticker
isin_ticker = pd.concat([monthly[["ISIN", "ticker"]], daily[["ISIN", "ticker"]]]).drop_duplicates().reset_index(drop=True)

### Getting the rows where the ISIN values are not unique
isin = isin_ticker[isin_ticker.duplicated(subset='ISIN', keep=False)].sort_index().sort_values(by='ISIN')

### If a row has a NaN value in the ticker column, then we keep the row with the ticker value
isin = isin.dropna(subset=['ticker'])

### Dropping the duplicate ISINs, but keeping the last one
isin = isin.drop_duplicates(subset='ISIN', keep='last')

### Creating a dictionary to map ISIN to ticker
isin_dict = isin.set_index('ISIN')['ticker'].to_dict()

### Updating the ticker column in the daily and monthly datasets
daily['ticker'] = daily['ISIN'].map(isin_dict).fillna(daily['ticker'])
monthly['ticker'] = monthly['ISIN'].map(isin_dict).fillna(monthly['ticker'])

### Getting all the unique tickers from both datasets
companies = pd.concat([daily, monthly])[['ticker', 'Name']].drop_duplicates(subset='ticker').sort_values('ticker').reset_index(drop=True)
companies

,ticker,Name
0,2020,2020 Bulkers
1,5PG,5th Planet Games
2,AASB,Aasen Sparebank
3,AAT,Aust-Agder Trafikkselskap
4,AAV,Adresseavisen
...,...,...
1072,ZAP,Zaptec
1073,ZENA,Zenith Energy
1074,ZENT,Zenitel
1075,ZONC,Zoncolan


In [5]:
# Ensure dates and tickers are in the right order
dates = daily['Date'].drop_duplicates().sort_values()
tickers = companies['ticker']

### Lets create the pivot tables
daily_returns = daily.pivot_table(
    index='Date',
    columns='ticker',
    values='Return',
    aggfunc='first'
).reindex(index=dates, columns=tickers)

daily_returns

ticker,2020,5PG,AASB,AAT,AAV,ABL,ABS,ABTEC,ACC,ACH,...,WWIN,XPLRA,XXL,YAR,ZAL,ZAP,ZENA,ZENT,ZONC,ZWIPE
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-04,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-08,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-08,-0.0104,-0.0074,0.0487,NaN,NaN,0.0000,0.0459,-0.0084,-0.0167,NaN,...,NaN,0.0073,-0.0125,0.0010,-0.0161,-0.0096,-0.0837,NaN,NaN,0.1591
2024-08-09,0.0197,0.0037,NaN,NaN,NaN,0.0131,-0.0263,0.0506,0.0146,NaN,...,NaN,0.0361,0.0000,0.0013,-0.0082,0.0019,0.0025,NaN,NaN,-0.0216
2024-08-12,0.0028,-0.0037,NaN,NaN,NaN,0.0345,0.0000,-0.0775,0.0192,NaN,...,NaN,-0.0209,0.0087,-0.0050,0.0193,0.0222,0.0481,NaN,NaN,-0.0060


In [6]:
# Ensure dates and tickers are in the right order
dates = monthly['Date'].drop_duplicates().sort_values()

### Lets turn the long format into wide format
monthly_shares = monthly.pivot_table(
    index='Date',
    columns='ticker',
    values='Shares',
    aggfunc='first'
).reindex(index=dates, columns=tickers)

monthly_shares

ticker,2020,5PG,AASB,AAT,AAV,ABL,ABS,ABTEC,ACC,ACH,...,WWIN,XPLRA,XXL,YAR,ZAL,ZAP,ZENA,ZENT,ZONC,ZWIPE
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-31,NaN,NaN,NaN,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-30,22620906.0,207960000.0,1700000.0,NaN,NaN,122936528.0,24425720.0,20673792.0,604200000.0,NaN,...,NaN,41700000.0,387944704.0,255000000.0,21600000.0,87520792.0,NaN,NaN,NaN,58413732.0
2024-05-31,22620906.0,207960000.0,1700000.0,NaN,NaN,122936528.0,24425720.0,20673792.0,604200000.0,NaN,...,NaN,41700000.0,387944704.0,255000000.0,21600000.0,87520792.0,NaN,NaN,NaN,58413732.0
2024-06-30,22620906.0,207960000.0,1700000.0,NaN,NaN,122936528.0,24425720.0,20673792.0,604200000.0,NaN,...,NaN,41700000.0,387944704.0,255000000.0,21600000.0,87520792.0,NaN,NaN,NaN,58413732.0


In [7]:
monthly_prices = monthly.pivot_table(
    index='Date',
    columns='ticker',
    values='Price',
    aggfunc='first'
).reindex(index=dates, columns=tickers)

monthly_prices

ticker,2020,5PG,AASB,AAT,AAV,ABL,ABS,ABTEC,ACC,ACH,...,WWIN,XPLRA,XXL,YAR,ZAL,ZAP,ZENA,ZENT,ZONC,ZWIPE
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-31,NaN,NaN,NaN,240.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,240.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,240.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,250.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,325.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-30,172.6,1.82,117.70,NaN,NaN,11.80,9.8,4.70,7.015,NaN,...,NaN,13.60,0.769,317.5,70.0,13.43,0.299,NaN,NaN,0.600
2024-05-31,163.5,1.60,111.00,NaN,NaN,12.50,10.3,3.83,7.415,NaN,...,NaN,12.80,0.700,324.9,77.6,13.58,0.285,NaN,NaN,0.596
2024-06-30,155.4,1.45,118.00,NaN,NaN,12.15,10.5,3.95,6.660,NaN,...,NaN,11.75,0.836,307.6,79.0,12.65,0.284,NaN,NaN,0.490


In [8]:
### Taking care of missing data
# ### Getting the tickers
# yf_ticker = [i + ".OL" for i in daily_prices.columns]

# ### Downloading the data
# yf_price = yf.download(yf_ticker, start='2020-01-01', end='2020-12-31')['Close']

# ### Removing the .OL from the columns
# yf_price.columns = [i.replace(".OL", "") for i in yf_price.columns]

# ### Making sure the index is a datetime
# yf_price.index = pd.to_datetime(yf_price.index)

# ### Saving the downloaded data as a csv
# yf_price.to_csv('Data/data_2020_yfinance.csv')

### Reading the data
yf_price = pd.read_csv('Data/missing_data/data_2020_yfinance.csv', index_col=0)

### Making sure the index is a datetime
yf_price.index = pd.to_datetime(yf_price.index)

### Getting the returns
yf_returns = yf_price.pct_change()

### Drop the first row
yf_returns = yf_returns.iloc[1:]

### Replace the 0 with NaN
yf_returns = yf_returns.replace(0, np.nan)

In [9]:
### Reading the data
bb_price = pd.read_csv('Data/missing_data/bloomberg_2020_2025.csv', index_col=0)

### Replacing the "#N/A Invalid Security" with NaN
bb_price = bb_price.replace('#N/A Invalid Security', np.nan)

### Making sure the index is a datetime
bb_price.index = pd.to_datetime(bb_price.index)

### Replacing " NO Equity" with "" in the columns
bb_price.columns = [i.replace(" NO Equity", "") for i in bb_price.columns]

### Getting the returns
bb_returns = bb_price.pct_change()
bb_returns = bb_returns.iloc[1:]

In [10]:
### Restricting the data to the correct period
bb_price = bb_price.loc['2020-07-01':'2020-11-30']
bb_returns = bb_returns.loc['2020-07-01':'2020-11-30']
yf_price = yf_price.loc['2020-07-01':'2020-11-30']
yf_returns = yf_returns.loc['2020-07-01':'2020-11-30']

### Using the bloomberg returns as a base, and then adding the yahoo finance returns where bloomberg is missing
returns_2020 = bb_returns.combine_first(yf_returns)
price_2020 = bb_price.combine_first(yf_price)

In [11]:
### Filling the index
idx = daily_returns.index.union(returns_2020.index)
daily_returns = daily_returns.reindex(idx)

### Filling the data
daily_returns.loc['2020-07-01':'2020-11-30'] = returns_2020

### Fixing the monthly prices
price_2020_monthly = price_2020.resample('M').last()

### Filling the index
idx = monthly_shares.index.union(price_2020_monthly.index)
monthly_shares = monthly_shares.reindex(idx)
monthly_prices = monthly_prices.reindex(idx)

### Filling the data
monthly_prices.loc['2020-07-01':'2020-11-30'] = price_2020_monthly

In [12]:
### Getting the dates where we are missing data
shares_monthly_2020 = monthly_shares.loc['2020-06-01':'2020-12-31']

### Looping trough it
for i in range(len(shares_monthly_2020.columns)):
    
    ### Monthly
    if not np.isnan(shares_monthly_2020.iloc[6, i]) or not np.isnan(shares_monthly_2020.iloc[6, i]):
        if not np.isnan(shares_monthly_2020.iloc[6, i]) and not np.isnan(shares_monthly_2020.iloc[0, i]):
            shares_monthly_2020.iloc[:, i] = shares_monthly_2020.iloc[:, i].interpolate()
        elif not np.isnan(shares_monthly_2020.iloc[6, i]):
            shares_monthly_2020.iloc[1:6, i] = shares_monthly_2020.iloc[6, i]
        elif not np.isnan(shares_monthly_2020.iloc[0, i]):
            shares_monthly_2020.iloc[1:6, i] = shares_monthly_2020.iloc[0, i]

### Filling the data
monthly_shares.loc['2020-06-01':'2020-12-31'] = shares_monthly_2020

### Getting the market cap
monthly_mcap = monthly_prices * monthly_shares

In [13]:
### Using the daily data to create the monthly data so we are sure they are consistent
monthly_returns = daily_returns.resample('M').apply(lambda x: (1 + x).prod() - 1)
monthly_returns = monthly_returns.replace(0, np.nan)

In [14]:
### Removing all columns that are only NaN or are dtypes object
monthly_returns = monthly_returns.dropna(axis=1, how='all')
monthly_prices = monthly_prices[monthly_returns.columns]
monthly_mcap = monthly_mcap[monthly_returns.columns]
monthly_shares = monthly_shares[monthly_returns.columns]

daily_returns = daily_returns[monthly_returns.columns]

In [15]:
### Connecting to the database
conn = sqlite3.connect('Data/data.db')

### Saving the unfiltred data in the database
daily_returns.to_sql('daily_returns', conn, if_exists='replace')
monthly_returns.to_sql('monthly_returns', conn, if_exists='replace')


### Closing the connection
conn.close()

In [16]:
filtered_monthly_returns = monthly_returns.copy()
filtered_daily_returns = daily_returns.copy()

for date in filtered_monthly_returns.index:
    month = date.strftime('%Y-%m')
    for company in filtered_monthly_returns.columns:
        if monthly_mcap.loc[date, company] < 1e6:
            filtered_daily_returns.loc[month, company] = np.nan
            filtered_monthly_returns.loc[date, company] = np.nan
        if monthly_mcap.loc[date, company] == np.nan or monthly_mcap.loc[date, company] < 1e7:
            if monthly_prices.loc[date, company] < 10:
                filtered_daily_returns.loc[month, company] = np.nan
                filtered_monthly_returns.loc[date, company] = np.nan

### Dropping all companies we have less then 3 months of data for
filtered_monthly_returns = monthly_returns.dropna(axis=1, thresh=6)

### If we dont have monthly returns for a company, then we remove it from the daily data
filtered_daily_returns = daily_returns[filtered_monthly_returns.columns]

In [17]:
print(f"We have {monthly_returns.shape[1]} tickers in the unfiltered monthly dataset")
print(f"We have {daily_returns.shape[1]} tickers in the unfiltered daily dataset")
print(f"We have {filtered_monthly_returns.shape[1]} tickers in the filtered monthly dataset")
print(f"We have {filtered_daily_returns.shape[1]} tickers in the filtered daily dataset")
print(f"We will only use the tickers that are in both filtered datasets, so we have {len(filtered_monthly_returns.columns)} tickers")

We have 1074 tickers in the unfiltered monthly dataset
We have 1074 tickers in the unfiltered daily dataset
We have 995 tickers in the filtered monthly dataset
We have 995 tickers in the filtered daily dataset
We will only use the tickers that are in both filtered datasets, so we have 995 tickers


In [18]:
### Connecting to the database
conn = sqlite3.connect('Data/data.db')

### Saving the filtred data in the database
filtered_daily_returns.to_sql('filtered_daily_returns', conn, if_exists='replace')
filtered_monthly_returns.to_sql('filtered_monthly_returns', conn, if_exists='replace')


### Closing the connection
conn.close()